# Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.nn.functional as F
torch.manual_seed(42)
np.random.seed(42)
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch_geometric.nn import knn_graph
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data load

In [2]:
expression_data = pd.read_csv('TCGA-SARC.star_tpm.tsv', sep='\t', index_col=0)  # Gene expression (TPM)
methylation_data = pd.read_csv('TCGA-SARC.methylation450.tsv', sep='\t', index_col=0)  # DNA methylation
copy_number_data = pd.read_csv('TCGA-SARC.gene-level_absolute.tsv', sep='\t', index_col=0)  # Gene-level copy number (ABSOLUTE)
protein_data = pd.read_csv('TCGA-SARC.protein.tsv', sep='\t', index_col=0)  # Protein expression

try:
    phenotype_data = pd.read_csv('TCGA-SARC.clinical.tsv', sep='\t', index_col=0)
except Exception as e:
    print("Error while loading phenotype_data:", e)
    with open('TCGA-SARC.clinical.tsv', 'r') as f:
        for i, line in enumerate(f):
            try:
                pd.read_csv(pd.compat.StringIO(line), sep='\t')
            except Exception as row_e:
                print(f"Error in line {i}: {row_e}")
    phenotype_data = pd.read_csv('TCGA-SARC.clinical.tsv', sep='\t', index_col=0, on_bad_lines='skip')

print("Data shapes:")
print(f"Expression data: {expression_data.shape}")
print(f"Methylation data: {methylation_data.shape}")
print(f"Copy number data: {copy_number_data.shape}")
print(f"Protein data: {protein_data.shape}")
print(f"Phenotype data: {phenotype_data.shape}")

Data shapes:
Expression data: (60660, 265)
Methylation data: (486427, 269)
Copy number data: (60623, 248)
Protein data: (487, 226)
Phenotype data: (272, 78)


# Sample Matching

In [3]:
# subtype_columns = [col for col in phenotype_data.columns if any(keyword in col.lower() 
#                    for keyword in ['subtype', 'histology', 'type', 'classification'])]

# print("Subtype-related columns:")
# for col in subtype_columns:
#     print(f"- {col}")
#     print(phenotype_data[col].value_counts())
#     print()


print("Checking for missing values in phenotype data:")
missing_values = phenotype_data.isnull().sum()
print(missing_values[missing_values > 0])

print()
print("Checking sample overlap between different omics data:")

samples_expression = set(expression_data.columns)
samples_methylation = set(methylation_data.columns)
samples_cnv = set(copy_number_data.columns)
samples_clinical = set(phenotype_data.index)
samples_protein = set(protein_data.columns)
print("Sample overlap:")
print(f"Expression samples: {len(samples_expression)}")
print(f"Methylation samples: {len(samples_methylation)}")
print(f"CNV samples: {len(samples_cnv)}")
print(f"Clinical samples: {len(samples_clinical)}")
print(f"Protein samples: {len(samples_protein)}")

# Find common samples across all omics
common_samples = list(samples_expression.intersection(samples_methylation, samples_cnv, samples_clinical, samples_protein))
print(f"Common samples across all omics: {len(common_samples)}")

# Filter data to keep only common samples
expression_data = expression_data[common_samples]
methylation_data = methylation_data[common_samples]
copy_number_data = copy_number_data[common_samples]
protein_data = protein_data[common_samples]
phenotype_data = phenotype_data.loc[common_samples]



Checking for missing values in phenotype data:
days_to_birth.demographic                                      1
year_of_birth.demographic                                      4
days_to_death.demographic                                    170
year_of_death.demographic                                    195
entity_submitter_id.annotations                              254
notes.annotations                                            254
submitter_id.annotations                                     254
classification.annotations                                   254
entity_id.annotations                                        254
created_datetime.annotations                                 254
annotation_id.annotations                                    254
entity_type.annotations                                      254
updated_datetime.annotations                                 254
case_id.annotations                                          254
state.annotations                          

# Data preprocessing

### Check Null value

In [4]:
#Check for null values of expression data
print("Checking for null values in expression data:")
null_expression = expression_data.isnull().sum().sum()
if null_expression > 0:
    print(f"Expression data contains {null_expression} null values.")
else:   
    print("No null values found in expression data.")
print()


# Check for null values in methylation data
print("Checking for null values in methylation data:")
null_methylation = methylation_data.isnull().sum().sum()
if null_methylation > 0:
    print(f"Methylation data contains {null_methylation} null values.")
else:
    print("No null values found in methylation data.")
print()


# Check for null values in copy number data
print("Checking for null values in copy number data:")
null_copy_number = copy_number_data.isnull().sum().sum()
if null_copy_number > 0:
    print(f"Copy number data contains {null_copy_number} null values.")
else:
    print("No null values found in copy number data.")
max_value = copy_number_data.max().max()
min_value = copy_number_data.min().min()

print(f"Maximum CNV value: {max_value}")
print(f"Minimum CNV value: {min_value}")
print()


# Check for null values in protein data
print("Checking for null values in protein data:")
null_protein = protein_data.isnull().sum().sum()
if null_protein > 0:
    print(f"Protein data contains {null_protein} null values.")
else:
    print("No null values found in protein data.")
print()


# Check for null values in phenotype data
print("Checking for null values in phenotype data:")
null_phenotype = phenotype_data.isnull().sum().sum()
if null_phenotype > 0:
    print(f"Phenotype data contains {null_phenotype} null values.")
else:
    print("No null values found in phenotype data.")



Checking for null values in expression data:
No null values found in expression data.

Checking for null values in methylation data:
Methylation data contains 16814049 null values.

Checking for null values in copy number data:
Copy number data contains 855388 null values.
Maximum CNV value: 7.0
Minimum CNV value: 0.0

Checking for null values in protein data:
Protein data contains 6300 null values.

Checking for null values in phenotype data:
Phenotype data contains 3389 null values.
Maximum CNV value: 7.0
Minimum CNV value: 0.0

Checking for null values in protein data:
Protein data contains 6300 null values.

Checking for null values in phenotype data:
Phenotype data contains 3389 null values.


### Process data

In [5]:
# expression data preprocessing -------------------------------------------------->

# Log2 transform
expression_data_log = np.log2(expression_data + 1)

# Z-score standardization
scaler_expr = StandardScaler()
expression_data_scaled = pd.DataFrame(
    scaler_expr.fit_transform(expression_data_log.T).T,
    index=expression_data_log.index,
    columns=expression_data_log.columns
)

from sklearn.feature_selection import VarianceThreshold


# Methylation data preprocessing -------------------------------------------------->

# Droping probes with more than 25% missing values 
methylation_data = methylation_data.dropna(thresh=0.75 * methylation_data.shape[1], axis=0)
# DO NOT fill na for MOFA
# methylation_data = methylation_data.apply(lambda x: x.fillna(x.median()), axis=1)

# Remove low-variance methylation probes
selector = VarianceThreshold(threshold=0.01)
methylation_filtered = pd.DataFrame(
    selector.fit_transform(methylation_data.T).T,
    index=methylation_data.index[selector.get_support()],
    columns=methylation_data.columns
)

# Z-score standardization
scaler_meth = StandardScaler()
methylation_scaled = pd.DataFrame(
    scaler_meth.fit_transform(methylation_filtered.T).T,
    index=methylation_filtered.index,
    columns=methylation_filtered.columns
)


# Copy number data preprocessing -------------------------------------------------->

# Drop genes with >20% missing values
gene_missing_threshold = 0.2
copy_number_data_filtered = copy_number_data.loc[
    copy_number_data.isnull().mean(axis=1) < gene_missing_threshold
]
# DO NOT fill null values for MOFA
# copy_number_imputed = copy_number_data_filtered.apply(
#     lambda row: row.fillna(row.median()), axis=1
# )
copy_number_imputed = copy_number_data_filtered

# Standardize across samples
scaler_cnv = StandardScaler()
copy_number_scaled = pd.DataFrame(
    scaler_cnv.fit_transform(copy_number_imputed.T).T,
    index=copy_number_imputed.index,
    columns=copy_number_imputed.columns
)


# Protein data preprocessing ------------------------------------------------------>
# Drop proteins with >20% missing values
protein_missing_threshold = 0.2
protein_data_filtered = protein_data.loc[
    protein_data.isnull().mean(axis=1) < protein_missing_threshold
]
# DO NOT fill null values for MOFA
# protein_imputed = protein_data_filtered.apply(
#     lambda row: row.fillna(row.median()), axis=1
# )
protein_imputed = protein_data_filtered

# Standardize across samples
scaler_protein = StandardScaler()
protein_scaled = pd.DataFrame(
    scaler_protein.fit_transform(protein_imputed.T).T,
    index=protein_imputed.index,
    columns=protein_imputed.columns
)

In [6]:
# Phenotype data preprocessing -------------------------------------------------->
subtype_column = 'primary_diagnosis.diagnoses'
print(f"Using '{subtype_column}' as the subtype column")
print(f"Subtype distribution:\n{phenotype_data[subtype_column].value_counts()}")
print()
# Only keep selected subtypes
selected_subtypes = [
    'Leiomyosarcoma, NOS',
    'Dedifferentiated liposarcoma',
    'Undifferentiated sarcoma',
    'Fibromyxosarcoma'
 ]
phenotype_data = phenotype_data[phenotype_data[subtype_column].isin(selected_subtypes)]
print(f"Filtered to selected subtypes. Remaining samples: {len(phenotype_data)}")
# Checking missing values in the subtype column
missing_subtypes = phenotype_data[subtype_column].isnull().sum()
print(f"Missing values in subtype column: {missing_subtypes}")
if missing_subtypes > 0:
    phenotype_data_clean = phenotype_data.dropna(subset=[subtype_column])
    print(f"Removed {missing_subtypes} samples with missing subtypes")
    print(f"Remaining samples: {len(phenotype_data_clean)}")
else:
    phenotype_data_clean = phenotype_data.copy()
    print("No missing subtypes found")

Using 'primary_diagnosis.diagnoses' as the subtype column
Subtype distribution:
primary_diagnosis.diagnoses
Leiomyosarcoma, NOS                        76
Dedifferentiated liposarcoma               49
Undifferentiated sarcoma                   32
Fibromyxosarcoma                           20
Malignant fibrous histiocytoma             11
Malignant peripheral nerve sheath tumor     9
Synovial sarcoma, spindle cell              4
Giant cell sarcoma                          3
Pleomorphic liposarcoma                     2
Synovial sarcoma, NOS                       1
Synovial sarcoma, biphasic                  1
Liposarcoma, well differentiated            1
Myxoid leiomyosarcoma                       1
Name: count, dtype: int64

Filtered to selected subtypes. Remaining samples: 177
Missing values in subtype column: 0
No missing subtypes found


### Sample matching

In [7]:
# Extract subtypes for the common samples
subtypes = phenotype_data_clean[subtype_column]
# Encode subtypes as numeric labels
label_encoder = LabelEncoder()
subtype_encoded = label_encoder.fit_transform(subtypes)
# Create mapping to encode subtype classes
subtype_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"\nSubtype encoding mapping:")
for subtype, encoded in subtype_mapping.items():
    print(f"  {encoded}: {subtype}")
# Converted to pandas Series to easily handle the index
subtype_encoded = pd.Series(subtype_encoded, index=subtypes.index, name='subtype_encoded')
print(f"\nEncoded subtype distribution:")
print(subtype_encoded.value_counts().sort_index())
# Update common samples with available subtypes
valid_samples = list(set(common_samples).intersection(set(phenotype_data_clean.index)))
print(f"\nUpdating common samples from {len(common_samples)} to {len(valid_samples)} (removed samples with missing subtypes or not in selected subtypes)")
expression_data_scaled = expression_data_scaled[valid_samples]
methylation_scaled = methylation_scaled[valid_samples]
copy_number_scaled = copy_number_scaled[valid_samples]
protein_scaled = protein_scaled[valid_samples]
subtype_encoded = subtype_encoded.loc[valid_samples]
common_samples = valid_samples
print()
print(f"Final data shapes after phenotype preprocessing:")
print(f"  Expression: {expression_data_scaled.shape}")
print(f"  Methylation: {methylation_scaled.shape}")
print(f"  Copy number: {copy_number_scaled.shape}")
print(f"  Protein: {protein_scaled.shape}")
print(f"  Phenotype: {phenotype_data_clean.shape}")
print(f"  Subtypes: {len(subtype_encoded)}")
print(f"  Common samples: {len(common_samples)}")


Subtype encoding mapping:
  0: Dedifferentiated liposarcoma
  1: Fibromyxosarcoma
  2: Leiomyosarcoma, NOS
  3: Undifferentiated sarcoma

Encoded subtype distribution:
subtype_encoded
0    49
1    20
2    76
3    32
Name: count, dtype: int64

Updating common samples from 210 to 177 (removed samples with missing subtypes or not in selected subtypes)

Final data shapes after phenotype preprocessing:
  Expression: (60660, 177)
  Methylation: (223630, 177)
  Copy number: (56646, 177)
  Protein: (457, 177)
  Phenotype: (177, 78)
  Subtypes: 177
  Common samples: 177


In [8]:
print("Expression:", expression_data_scaled.shape)
print("Methylation:", methylation_scaled.shape)
print("Copy Number:", copy_number_scaled.shape)
print("Protein:", protein_scaled.shape)
print("Phenotype:", phenotype_data_clean.shape)
print("Subtype labels:", subtype_encoded.shape)
print("Common samples:", len(common_samples))

Expression: (60660, 177)
Methylation: (223630, 177)
Copy Number: (56646, 177)
Protein: (457, 177)
Phenotype: (177, 78)
Subtype labels: (177,)
Common samples: 177


In [ ]:

from mofapy2.run.entry_point import entry_point

# Prepare omics data for MOFA (features x samples, all must have same columns)
omics_views = [
    expression_data_scaled,
    methylation_scaled,
    copy_number_scaled,
    protein_scaled
 ]
omics_views = [df.select_dtypes(include=[float, int]) for df in omics_views]
omics_views = [df[common_samples] for df in omics_views]
# Convert to numpy arrays (features x samples)
data_list = [view.values for view in omics_views]
data_list = [data_list]  # Wrap in a list for 1 group with 4 views
for i, arr in enumerate(data_list[0]):
    print(f"View {i} shape: {arr.shape}")
# Initialize MOFA
ent = entry_point()
ent.set_data_options(scale_groups=False, scale_views=False)
ent.set_data_matrix(data_list)
ent.set_model_options(factors=20, spikeslab_factors=True, ard_factors=True, ard_weights=True)
ent.set_train_options(iter=50, convergence_mode="medium", seed=42, verbose=True)
ent.build()
ent.run()
model = ent.model
# Extract factors (Z) and weights (W)
factors = model.nodes["Z"].getExpectation() #list(one per group):each is samples x factors
weights = model.nodes["W"].getExpectation() #list(one per view):each is features x factors
print("MOFA factors shape (samples x factors):", factors[0].shape)
print("MOFA weights shape (features x factors):", weights[0].shape)

View 0 shape: (60660, 177)
View 1 shape: (223630, 177)
View 2 shape: (56646, 177)
View 3 shape: (457, 177)

        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
View names not provided, using default naming convention:
- view1, view2, ..., viewM

Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

S